In [17]:
import threading

class ReaderWriterLock:
    
    def __init__(self):
        self.readers = 0
        self.writers = 0
        self.rwlock = threading.Lock()
        self.read_ready = threading.Condition(self.rwlock)
        self.write_ready = threading.Condition(self.rwlock)
    
    def rlock(self):
        self.rwlock.acquire()
        
        while self.writers > 0:
            self.read_ready.wait()
        self.readers += 1
        
        self.rwlock.release()
    
    def runlock(self):
        self.rwlock.acquire()
        
        self.readers -= 1
        self.signal_next()
        
        self.rwlock.release()
    
    def wlock(self):
        self.rwlock.acquire()
        self.writers += 1
        
        if self.writers > 1 or self.readers > 0:
            self.write_ready.wait()
        
        self.rwlock.release()
    
    def wunlock(self):
        self.rwlock.acquire()
        
        self.writers -= 1
        self.signal_next()
        
        self.rwlock.release()
    
    def signal_next(self):
        if self.writers > 0 and self.readers == 0:
            self.write_ready.notify()
        
        if self.writers == 0:
            self.read_ready.notifyAll()

In [18]:
import time
rwl = ReaderWriterLock()
class Reader(threading.Thread):
    
    def run(self):
        print('Read Lock Requested ')
        rwl.rlock()
        print('Read Lock Acquired ')
        time.sleep(5)
        rwl.runlock()
        print('Read Lock Released ')

class Writer(threading.Thread):
    
    def run(self):
        print('Write Lock Requested ')
        rwl.wlock()
        print('Write Lock Acquired ')
        time.sleep(10)
        rwl.wunlock()
        print('Write Lock Released ')

Reader().start()
Reader().start()
Writer().start()
time.sleep(1)
Reader().start()
Reader().start()
Reader().start()
Writer().start()

Read Lock Requested 
Read Lock Acquired 
Read Lock Requested 
Read Lock Acquired 
Write Lock Requested 
Read Lock Requested 
Read Lock Requested 
Read Lock Requested 
Write Lock Requested 
Read Lock Released 
Read Lock Released Write Lock Acquired 

Write Lock Released 
Write Lock Acquired 
Write Lock Released Read Lock Acquired 

Read Lock Acquired 
Read Lock Acquired 
Read Lock Released Read Lock Released Read Lock Released 


